## Обучение с подкреплением
![Img](https://raw.githubusercontent.com/DPritykin/Control-Theory-Course/main/Term%202%20Nonlinear%20Dynamics%20and%20Control/Images/rl.png)

<font size="3">Термин <b>подкрепление (reinforcement)</b>  пришёл из поведенческой психологии и обозначает награду или наказание за некоторый получившийся результат, зависящий не только от самих принятых решений, но и внешних, не обязательно подконтрольных, факторов. Под обучением здесь понимается поиск способов достичь желаемого результата методом проб и ошибок (trial and error), то есть попыток решить задачу и использование накопленного опыта для усовершенствования своей стратегии в будущем.</font>

<font size="3">В лекции будут введены основные определения и описана формальная постановка задачи. Под желаемым результатом мы далее будем понимать максимизацию некоторой скалярной величины, называемой <b> вознаграждением (reward)</b> . Интеллектуальную сущность (систему/робота/алгоритм), принимающую решения, будем называть <b> агентом (agent)</b> . Агент взаимодействует с миром (world) или средой (environment), которая задаётся зависящим от времени <b> состоянием (state)</b> . Агенту в каждый момент времени в общем случае доступно только некоторое <b> наблюдение (observation)</b>  текущего состояния мира. Сам агент задаёт процедуру выбора <b> действия (action) </b> по доступным наблюдениям; эту процедуру далее будем называть <b> стратегией (policy)</b> . Процесс взаимодействия агента и среды определяется <b> динамикой среды (world dynamics)</b> , включающей правила смены состояний среды во времени и генерации вознаграждения.</font>

<font size="3">Буквы $\mathbf{s}$, $\mathbf{a}$, $\mathbf{r}$ зарезервируем для состояний, действий и наград соответственно; буквой t будем обозначать время в процессе взаимодействия</font>

Во многом изложение будет следовать конспекту
[Sergey Ivanov Reinforcement Learning Textbook. (Конспект по обучению с подкреплением)](https://arxiv.org/abs/2201.09746)

### Марковская динамическая система

Пусть заданы:

1. Пространство возможных состояний: $s \in \mathcal{S}$;
    
2. Пространство управляющий воздействий (или действий): $a \in \mathcal{A}$;
    
3. Оператор эволюции системы 
$$
    \begin{equation*}
    T: \mathcal{S} \times \mathcal{A} \times \mathcal{S'} \rightarrow [0,1]
    \end{equation*}
$$
    (ставит в соответствие составительную и действительную вероятность перехода в следующее действие $s'\gets p(s'|s,a)$);
    
4. Вознаграждение
$$
    \begin{equation*}
    R: \mathcal{S} \times \mathcal{A} \times \mathcal{S'} \rightarrow \mathcal{R};
    \end{equation*}
$$

5. $\gamma \in [0,1)$ - степенной показатель для кумулятивной награды:
$$
    \begin{equation}
         R = \sum_{t=0}^{N} \gamma^t r_t
    \end{equation}
$$

Эта терминология применяется для математического описания алгоритмов взаимодействия агента со средой, имеющих целью обучение агента стратегиям, ведущим к максимальному вознаграждению:
$$
\pi(s, a) = p(a | s) = \text{arg} \max_{\pi} E[R]
$$

Агент взаимодействует со средой, выбирая \textit{действия} и получая по обратной связи состояния и \textit{вознаграждения}. Исходя из вознаграждений он "понимает", правильное ли действие он выбрал и таким образом обучается.


### V-функция и Q-функция

Также мы вводим:


1. Функцию полезности (V-функцию):
$$
    \begin{equation}
            V(s) = E \left (\sum_{t=0}^{\infty} \gamma^t r_t | \pi(s),\, s_0 = s \right)
    \end{equation}
$$
(ожидаемое вознаграждение, которое мы получим при старте из состояния $s$, где:
* \(\pi(s)\) - стратегия принятия решений (вероятность выбрать действие $a$, находясь в состоянии $s$);
* E - математическое ожидание.
        
2. Функцию качества (Q-функцию):
$$
    \begin{equation}
    Q(s,a) = \sum_{s'} T(s,a,s')(R(s,a,s') + \gamma V(s'))
    \end{equation}
$$    
(ожидаемое вознаграждение, которое мы получим, если в начальном в состоянии $s$, выбрали действие $a$)

### Принцип оптимальности Беллмана для value-функций

Для функции $V(s)$ принцип оптимальности Беллмана выглядит так:

$$
\begin{equation}\label{eq:bellmanV}
V(s) = \max{ \sum_{s'} T(s,a,s')(R(s,a,s') + \gamma V(s'))}
\end{equation}
$$
(если мы находимся в состоянии s, мы выбираем лучшее действие)

Принцип оптимальности Беллмана позволяет получать значения $V(s)$ при переходе от $s'$ к $s$ (от следующего состояния к предыдущему), если известна $V(s')$.

Для Q-функции:
$$
\begin{equation}\label{eq:bellmanQ}
Q(s,a) = \sum_{s'} T(s,a,s')(R(s,a,s') + \gamma \max_{a'} Q(s',a'))
\end{equation}
$$

### Простейшая среда GridWorld

![Img](https://raw.githubusercontent.com/DPritykin/Control-Theory-Course/main/Term%202%20Nonlinear%20Dynamics%20and%20Control/Images/gridworld.png)

<font size="3">  Агент начинает игру в произвольном свободном положении(на примере слева снизу поля) и заканчивает в ячейке с  +1 ("сектор приз") либо -1 ("сектор штраф"). 
    На каждом шагу он может двигаться вертикально или горизонтально, но не может проходить через препятствия или уходить за пределы игрового поля.</font>

#### Детерминированная версия:
</font> <font size="3">Подразумевается, что действие на каждом шаге детерминировано, то есть если агент выбрал действие "переместиться в клетку справа", то именно такое действие и реализуется (нет случайных факторов, которые могут повлиять на выполнение команды).</font>

#### Недетерминированная версия
</font> <font size="3">Реализация выбранного действия на каждом шаге не детерминирована, то есть агент, принимая решение перейти на соседнюю клеточку (например, наверх), оказывается на ней с вероятностью 0.8, оставшаяся вероятность равномерно распределяется между перпендикулярными направлениями движения (то есть, если агент принял решение пойти наверх, то перпендикулярные направления - вправо-влево).</font>

### Пример 1

[Реализация игры Grid World с помощью Value-iteration](https://github.com/DPritykin/Control-Theory-Course/blob/main/Term%202%20Nonlinear%20Dynamics%20and%20Control/Homework%20Problems/RL/Grid%20World%20Value-iteration.ipynb)

### Пример 2
[Реализация игры Grid World с помощью Q-learning](https://github.com/DPritykin/Control-Theory-Course/blob/main/Term%202%20Nonlinear%20Dynamics%20and%20Control/Homework%20Problems/RL/Grid%20World%20Q-learning.ipynb)